In [1]:
import cohere  
from openai import OpenAI
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlencode
import re
import pandas as pd


In [69]:
client = OpenAI(
    api_key="sk-dP3OLsFmsPb2kgfFaeYO5im3fx9KZkLtVBsMwRqMwvzsdQtI",  base_url="https://api.chatanywhere.tech"# This is the default and can be omitted
)

In [62]:
query = """In the TV show The Office, who said "out of your hands" and in which episode and season"""
query = """In the TV show The Office, who said "no more sex" and in which episode and season"""
query = """In the TV show The Office, who said "who do you think you are" and in which episode and season"""

In [63]:
encoded_query = urlencode({"q": query})
encoded_query

'q=In+the+TV+show+The+Office%2C+who+said+%22who+do+you+think+you+are%22+and+in+which+episode+and+season'

In [64]:
#maybe free to use google query, but the content is really ill-formated.
url =f"https://www.google.com/search?{encoded_query}&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRigATIHCAQQIRigATIHCAUQIRiPAjIHCAYQIRiPAtIBCDIxODlqMGo3qAIAsAIA&sourceid=chrome&ie=UTF-8&gl=us&hl=en"

In [65]:
try:
    # Fetch the webpage
    response = requests.get(url)
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract and return the text content
    content = soup.get_text(separator="\n").strip()
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")


In [66]:
# Function to clean up the data
def clean_data(text):
    # Remove newline characters and excess spaces
    text = re.sub(r'\s+', ' ', text)  # Replaces multiple spaces and newlines with one space
    # Remove unwanted characters like \n, \t, or other non-printable characters
    text = re.sub(r'[\n\t\r]+', ' ', text)
    # Additional cleaning if necessary (e.g., remove '×' and other unwanted symbols)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Removes non-ASCII characters
    return text.strip()

content = clean_data(content)

In [67]:
content

'In the TV show The Office, who said "who do you think you are" and in which episode and season - Google Search G o o g l e   Please click here if you are not redirected within a few seconds. All Videos Images Books Maps News Shopping Search tools Any time Any time Past hour Past 24 hours Past week Past month Past year All results All results Verbatim Who do you think you are? From Season 4 Episode 14 : Goodbye Toby - It\'s Toby\'s (Paul Lieberstein) goodbye party at Dunder Mifflin and Michael (Steve Carell) demands a huge celebration that matches the joy in his heart. Jan 18, 2022 i\'ll kill you - The Office US - YouTube www.youtube.com   watch About Featured Snippets Steve Carell as Michael Scott - Goodbye, Toby (2008) - IMDb www.imdb.com   title   characters ... The Office" Goodbye, Toby (TV Episode 2008) Steve Carell as Michael Scott. ... Michael Scott : Umm, who do you think you are? Toby Flenderson : [long pause] ... Paul Lieberstein as Toby Flenderson - IMDb www.imdb.com   title

In [71]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "don't use your own knowledge. use the data I gave you",
        },
        {
            "role": "user",
            "content": f"""based on {content}, answer the question {query}, 
             show the original line,
             list all answers""",
        }
    ],
    model="gpt-4o",
)
response1 = chat_completion.choices[0].message.content
response1

'In the TV show The Office, the line "Who do you think you are?" is said by Michael Scott, played by Steve Carell, in Season 4, Episode 14, titled "Goodbye Toby." The original line is part of a conversation where Michael Scott asks Toby Flenderson, played by Paul Lieberstein, "Umm, who do you think you are?" and Toby responds after a long pause, "I\'m Toby." Michael then confirms, "Yeah. Correct. Umm..."'

In [88]:
example_json = {"season":3, "episode":2,"character":"Andy", "text": ""}
names = ['Michael', 'Jim', 'Pam', 'Dwight', 'Jan', 'Michel', 'Todd Packer',
       'Phyllis', 'Stanley', 'Oscar', 'Angela', 'Kevin', 'Ryan', 'Man',
       'Roy', 'Mr. Brown', 'Toby', 'Kelly', 'Meredith', 'Travel Agent',
       'Man on Phone', 'Everybody', 'Lonny', 'Darryl']

In [89]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """don't use your own knowledge, rely on my data""",
        },
        {
            "role": "user",
            "content": f"""based on {response1},
            convert the answer into json object,
            just return the json, don't add json on the front.
            the name should be from {names}
            for example {example_json}""",
        }
    ],
    model="gpt-4o",
)
response2 = chat_completion.choices[0].message.content
response2 = json.loads(response2)
print(response2)

{'season': 4, 'episode': 14, 'character': 'Michael', 'text': 'Umm, who do you think you are?'}


In [77]:
df = pd.read_csv("../backend/schrute.csv", header=0)
df = df.drop("Unnamed: 0", axis=1)
print(df.columns)

df['lower'] = df['text'].str.lower()
df['lower'] = df['lower'].str.strip()
# Replace empty strings with NaN and then drop those rows
df.replace(pd.NA, "", inplace=True)

Index(['index', 'season', 'episode', 'episode_name', 'director', 'writer',
       'character', 'text', 'text_w_direction'],
      dtype='object')


In [93]:
candidate_text= df[(df['season'] == response2['season']) & (df['episode'] ==
                                            response2['episode']) & (df['character'] == response2['character'])]['text'].tolist()

In [97]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """don't use your own knowledge, rely on my data""",
        },
        {
            "role": "user",
            "content": f"""the query is {response2['text']},
             find the most relatable text from this array of text {candidate_text},
             then return top 3 most related text
            """,
        }
    ],
    model="gpt-4o",
)
response3 = chat_completion.choices[0].message.content
# response3 = json.loads(response2)
print(response3)

To find the text most relatable to the query "Umm, who do you think you are?," we should look for statements that touch on identity, self-perception, or challenge. Here are the top three related texts:

1. "What is wrong with these people? I would do anything for them and they're just hanging me out to dry."
2. "Dating shouldn't be hard for somebody like me, but it is and you know why? Because nobody here is willing to help me. Nothing would ever get done in this office without a formal request, would it? Well, fine. Here goes."
3. "Now you ruined it." 

These texts involve a sense of personal identity being questioned or challenged.


In [98]:
# this is a failure

# section 2

In [123]:
def get_google_search(url):
    try:
        # Fetch the webpage
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract and return the text content
        content = soup.get_text(separator="\n").strip()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    return content


def clean_data(text):
    # Remove newline characters and excess spaces
    # Replaces multiple spaces and newlines with one space
    text = re.sub(r'\s+', ' ', text)
    # Remove unwanted characters like \n, \t, or other non-printable characters
    text = re.sub(r'[\n\t\r]+', ' ', text)
    # Additional cleaning if necessary (e.g., remove '×' and other unwanted symbols)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Removes non-ASCII characters
    return text.strip()

def call_openai(client, messages, model="gpt-4o"):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
    )
    return chat_completion.choices[0].message.content


def is_valid_json_with_fields(string):
    try:
        # Try parsing the string to a Python dictionary
        data = json.loads(string)

        # Check if the required fields exist and have the correct types
        if isinstance(data, dict):
            if "season" in data and isinstance(data["season"], int) and \
               "episode" in data and isinstance(data["episode"], int) and \
               "character" in data and isinstance(data["character"], str) and \
               "text" in data and isinstance(data["text"], str):
                return True
            else:
                return False
        else:
            return False
    except json.JSONDecodeError:
        return False

In [130]:
def get_res_from_openai(query, example_json, retry=3):
    client = OpenAI(
        # This is the default and can be omitted
        api_key="sk-dP3OLsFmsPb2kgfFaeYO5im3fx9KZkLtVBsMwRqMwvzsdQtI",  base_url="https://api.chatanywhere.tech"
    )
    
    encoded_query = urlencode({"q": query})
    #maybe free to use google query, but the content is really ill-formated.
    url =f"https://www.google.com/search?{encoded_query}&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRigATIHCAQQIRigATIHCAUQIRiPAjIHCAYQIRiPAtIBCDIxODlqMGo3qAIAsAIA&sourceid=chrome&ie=UTF-8&gl=us&hl=en"
    content = get_google_search(url)
    content = clean_data(content)
    
    prompt1 = [
        {
            "role": "system",
            "content": "don't use your own knowledge. use the data I gave you",
        },
        {
            "role": "user",
            "content": f"""based on {content}, answer the question {query}, list all answers""",
        }
    ]
    
    res1 = call_openai(client, prompt1, "gpt-4o-mini")
    
    prompt2 = [
        {
            "role": "system",
            "content": """don't use your own knowledge, rely on my data""",
        },
        {
            "role": "user",
            "content": f"""based on {res1},
            convert the answer into json object, just return the json, don't add json on the front.
            for example {example_json}""",
        }
    ]
    
    res2 = call_openai(client, prompt2)
    # for i in range(retry):
    #     res2 = call_openai(client, prompt2)
    #     if is_valid_json_with_fields(res2):
    return json.loads(res2), content, res1, res2
    

In [131]:
query = """In the TV show The Office, who said "out of your hands" and in which episode and season"""
# query = """In the TV show The Office, who said "no more sex" and in which episode and season"""
# query = """In the TV show The Office, who said "who do you think I am" and in which episode and season"""

example_json = {"season": 3, "episode": 2, "character": "Andy", "text": ""}

In [132]:
res, content, response1, response2 = get_res_from_openai(
    query=query, example_json=example_json)
print(res)

{'season': 1, 'episode': 1, 'character': 'Stanley', 'text': 'It could be out of your hands, Michael.'}


In [133]:
content

'In the TV show The Office, who said "out of your hands" and in which episode and season - Google Search G o o g l e   Please click here if you are not redirected within a few seconds. All Images Videos Books Maps News Shopping Search tools Any time Any time Past hour Past 24 hours Past week Past month Past year All results All results Verbatim The Office (USA)   Season 1 Episode 1: "Pilot" | Genius genius.com   The-office-usa-season-1-episode-1-pilot-annotated OSCAR: Yeah, but Michael what if they downsize here? MICHAEL: Not gonna happen. STANLEY: It could be out of your hands, Michael MICHAEL: It won\'t be out of ... People also ask What happens in season 2 episode 12 of The Office? What happens in season 3 episode 23 of The Office? What happens in season 5 episode 27 of The Office? What happens in season 3 episode 14 of The Office? Pilot - The Office TV Show (Season 1) the-office-tv-show.com   episodes   season1   pilot It could be out of your hands Michael. It won\'t be out of my h

In [28]:
df = pd.read_csv("../backend/schrute.csv", header=0)
df = df.drop("Unnamed: 0", axis=1)
print(df.columns)

df['lower'] = df['text'].str.lower()
df['lower'] = df['lower'].str.strip()
# Replace empty strings with NaN and then drop those rows
df.replace(pd.NA, "", inplace=True)

Index(['index', 'season', 'episode', 'episode_name', 'director', 'writer',
       'character', 'text', 'text_w_direction'],
      dtype='object')


In [87]:
# df.character.unique()

In [41]:
res

{'season': 3,
 'episode': 19,
 'character': 'Michael Scott',
 'text': 'no more sex.'}

In [116]:
# df[(df['season'] == res['season']) & (df['episode'] ==
#                                       res['episode']) & (df['character'] == "Michael")]